In [4]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Hypothesis Testing

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Our null hypothesis is that both university and non-university towns are affected equally. We will now run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files is used:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [28]:
# this dictionary maps state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [29]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needed to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    uni = pd.read_csv('university_towns.txt',sep="\r", header=None)
    uni['State'] = uni[0].apply(lambda x: x.split('[')[0].strip() if x.count('[')>0 and x.count('(')==0 else np.NaN) 
    uni['RegionName'] = uni[0].apply(lambda x: x.split('(')[0].strip() if x.count('(')>0 else x)
    uni = uni.drop(0,axis=1)
    uni['State'] = uni['State'].fillna(method='ffill')
    uni = uni.drop(uni[uni['RegionName'].str.contains('[', regex=False)].index)
    uni = uni.reset_index(drop=True)

    
    return uni

In [22]:
# stateStr = """
# Ohio, Kentucky, American Samoa, Nevada, Wyoming
# ,National, Alabama, Maryland, Alaska, Utah
# ,Oregon, Montana, Illinois, Tennessee, District of Columbia
# ,Vermont, Idaho, Arkansas, Maine, Washington
# ,Hawaii, Wisconsin, Michigan, Indiana, New Jersey
# ,Arizona, Guam, Mississippi, Puerto Rico, North Carolina
# ,Texas, South Dakota, Northern Mariana Islands, Iowa, Missouri
# ,Connecticut, West Virginia, South Carolina, Louisiana, Kansas
# ,New York, Nebraska, Oklahoma, Florida, California
# ,Colorado, Pennsylvania, Delaware, New Mexico, Rhode Island
# ,Minnesota, Virgin Islands, New Hampshire, Massachusetts, Georgia
# ,North Dakota, Virginia
# """
# #list of regionName entries string length
# regNmLenStr = """
# 06,08,12,10,10,04,10,08,09,09,05,06,11,06,12,09,08,10,12,06,
# 06,06,08,05,09,06,05,06,10,28,06,06,09,06,08,09,10,35,09,15,
# 13,10,07,21,08,07,07,07,12,06,14,07,08,16,09,10,11,09,10,06,
# 11,05,06,09,10,12,06,06,11,07,08,13,07,11,05,06,06,07,10,08,
# 11,08,13,12,06,04,08,10,08,07,12,05,06,09,07,10,16,10,06,12,
# 08,07,06,06,06,11,14,11,07,06,06,12,08,10,11,06,10,14,04,11,
# 18,07,07,08,09,06,13,11,12,10,07,12,07,04,08,09,09,13,08,10,
# 16,09,10,08,06,08,12,07,11,09,07,09,06,12,06,09,07,10,09,10,
# 09,06,15,05,10,09,11,12,10,10,09,13,06,09,11,06,11,09,13,37,
# 06,13,06,09,49,07,11,12,09,11,11,07,12,10,06,06,09,04,09,15,
# 10,12,05,09,08,09,09,07,14,06,07,16,12,09,07,09,06,32,07,08,
# 08,06,10,36,09,10,09,06,09,11,09,06,10,07,14,08,07,06,10,09,
# 05,11,07,06,08,07,05,07,07,04,06,05,09,04,25,06,07,08,05,08,
# 06,05,11,09,07,07,06,13,09,05,16,05,10,09,08,11,06,06,06,10,
# 09,07,06,07,10,05,08,07,06,08,06,30,09,07,06,11,07,12,08,09,
# 16,12,11,08,06,04,10,10,15,05,11,11,09,08,06,04,10,10,07,09,
# 11,08,26,07,13,05,11,03,08,07,06,05,08,13,10,08,08,08,07,07,
# 09,05,04,11,11,07,06,10,11,03,04,06,06,08,08,06,10,09,05,11,
# 07,09,06,12,13,09,10,11,08,07,07,08,09,10,08,10,08,56,07,12,
# 07,16,08,04,10,10,10,10,07,09,08,09,09,10,07,09,09,09,12,14,
# 10,29,19,07,11,12,13,13,09,10,12,12,12,08,10,07,10,07,07,08,
# 08,08,09,10,09,11,09,07,09,10,11,11,10,09,09,12,09,06,08,07,
# 12,09,07,07,06,06,08,06,15,08,06,06,10,10,10,07,05,10,07,11,
# 09,12,10,12,04,10,05,05,04,14,07,10,09,07,11,10,10,10,11,15,
# 09,14,12,09,09,07,12,04,10,10,06,10,07,28,06,10,08,09,10,10,
# 10,13,12,08,10,09,09,07,09,09,07,11,11,13,08,10,07
# """

# df = get_list_of_university_towns()

# cols = ["State", "RegionName"]

# print('Shape test: ', "Passed" if df.shape ==
#       (517, 2) else 'Failed')
# print('Index test: ',
#       "Passed" if df.index.tolist() == list(range(517))
#       else 'Failed')

# print('Column test: ',
#       "Passed" if df.columns.tolist() == cols else 'Failed')
# print('\\n test: ',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\n')) or any(df[cols[1]].str.contains('\n'))
#       else 'Passed')
# print('Trailing whitespace test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\s+$')) or any(df[cols[1]].str.contains(
#               '\s+$'))
#       else 'Passed')
# print('"(" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\(')) or any(df[cols[1]].str.contains(
#               '\('))
#       else 'Passed')
# print('"[" test:',
#       "Failed" if any(df[cols[0]].str.contains(
#           '\[')) or any(df[cols[1]].str.contains(
#               '\]'))
#       else 'Passed')

# states_vlist = [st.strip() for st in stateStr.split(',')]

# mismatchedStates = df[~df['State'].isin(
#     states_vlist)].loc[:, 'State'].unique()
# print('State test: ', "Passed" if len(
#     mismatchedStates) == 0 else "Failed")
# if len(mismatchedStates) > 0:
#     print()
#     print('The following states failed the equality test:')
#     print()
#     print('\n'.join(mismatchedStates))

# df['expected_length'] = [int(s.strip())
#                          for s in regNmLenStr.split(',')
#                          if s.strip().isdigit()]
# regDiff = df[df['RegionName'].str.len() != df['expected_length']].loc[
#     :, ['RegionName', 'expected_length']]
# regDiff['actual_length'] = regDiff['RegionName'].str.len()
# print('RegionName test: ', "Passed" if len(regDiff) ==
#       0 else ' \nMismatching regionNames\n {}'.format(regDiff))

In [30]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    recess = pd.read_excel('gdplev.xls',header = 0,skiprows = [0,1,2,3,4,6,7]).drop(['Unnamed: 0','Unnamed: 3','Unnamed: 7',
                                                                                      'GDP in billions of current dollars',
                                                                                      'GDP in billions of chained 2009 dollars',
                                                                                      'GDP in billions of current dollars.1'],
                                                                                     axis=1)                                                                       

    recess = recess.rename(columns={'GDP in billions of chained 2009 dollars.1':'GDP in billions of chained 2009 dollars.',
                                'Unnamed: 4':'Quarter'}).drop(recess.iloc[0:212].index).reset_index(drop=True)
    GDP_val = list()

    for i in range(len(recess['GDP in billions of chained 2009 dollars.'])-2):
        if recess['GDP in billions of chained 2009 dollars.'].iloc[i+1]<recess['GDP in billions of chained 2009 dollars.'].iloc[i] and recess['GDP in billions of chained 2009 dollars.'].iloc[i+2]<recess['GDP in billions of chained 2009 dollars.'].iloc[i+1]:
            GDP_val.append(recess['GDP in billions of chained 2009 dollars.'].iloc[i+1])
        
    start_recess = recess[recess['GDP in billions of chained 2009 dollars.']==GDP_val[0]].values[0][0]
    
    return start_recess
get_recession_start()

'2008q3'

In [31]:
def get_recession_end():
    recess = pd.read_excel('gdplev.xls',header = 0,skiprows = [0,1,2,3,4,6,7]).drop(['Unnamed: 0','Unnamed: 3',
                                                                                    'Unnamed: 7',
                                                                                 'GDP in billions of current dollars',
                                                                                 'GDP in billions of chained 2009 dollars',
                                                                                'GDP in billions of current dollars.1'],
                                                                                axis=1)

    recess = recess.rename(columns={'GDP in billions of chained 2009 dollars.1':'GDP in billions of chained 2009 dollars.',
                                'Unnamed: 4':'Quarter'}).drop(recess.iloc[0:212].index).reset_index(drop=True)
    start_recess = get_recession_start()
    start_recess_indx = recess[recess['Quarter']==start_recess].index[0]
    GDP_val = list()

    for i in range(start_recess_indx,len(recess['GDP in billions of chained 2009 dollars.'])-2):
        if recess['GDP in billions of chained 2009 dollars.'].iloc[i+1]>recess['GDP in billions of chained 2009 dollars.'].iloc[i] and recess['GDP in billions of chained 2009 dollars.'].iloc[i+2]>recess['GDP in billions of chained 2009 dollars.'].iloc[i+1]:
            GDP_val.append(recess['GDP in billions of chained 2009 dollars.'].iloc[i+2])
        
    end_recess = recess[recess['GDP in billions of chained 2009 dollars.']==GDP_val[0]].values[0][0]
      
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    return end_recess
get_recession_end()

'2009q4'

In [32]:
def get_recession_bottom():
    recess = pd.read_excel('gdplev.xls',header = 0,skiprows = [0,1,2,3,4,6,7]).drop(['Unnamed: 0','Unnamed: 3',
                                                                                    'Unnamed: 7',
                                                                                 'GDP in billions of current dollars',
                                                                                 'GDP in billions of chained 2009 dollars',
                                                                                'GDP in billions of current dollars.1'],
                                                                                axis=1)

    recess = recess.rename(columns={'GDP in billions of chained 2009 dollars.1':'GDP in billions of chained 2009 dollars.',
                                'Unnamed: 4':'Quarter'}).drop(recess.iloc[0:212].index).reset_index(drop=True)
    start_recess = get_recession_start()
    start_recess_indx = recess[recess['Quarter']==start_recess].index[0]
    
    end_recess = get_recession_end()
    end_recess_indx = recess[recess['Quarter']==end_recess].index[0]
    
    recess_time = list()
    
    for i in range(start_recess_indx, end_recess_indx+1):
        recess_time.append(recess['GDP in billions of chained 2009 dollars.'].iloc[i])
    
    recess_bottom_val = min(recess_time)
    recess_bottom_Q = recess[recess['GDP in billions of chained 2009 dollars.']==recess_bottom_val].values[0][0]
        
    
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    return recess_bottom_Q
get_recession_bottom()

'2009q2'

In [33]:
def convert_housing_data_to_quarters():
    housing_data = pd.read_csv('City_Zhvi_AllHomes.csv')
    housing_data.drop(['RegionID','Metro','CountyName','SizeRank'],axis=1, inplace = True)
    start_drop = pd.Period('1996-04')
    end_drop = pd.Period('1999-12')
    drop_range = pd.period_range(start_drop, end_drop, freq = 'M').strftime('%Y-%m')
    housing_data = housing_data.drop(drop_range,axis=1)
    housing_data['State'] = housing_data['State'].replace(states)

    housing_data.set_index(['State','RegionName'],inplace=True)
    start_date = pd.Period('2000-01')
    end_date = pd.Period('2016-08')
    time_span = pd.period_range(start_date,end_date,freq="M")
    housing_data.columns = time_span
    housing_data = housing_data.resample('Q',axis=1).mean()
    housing_data.columns = housing_data.columns.strftime('%Yq%q')
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q, with a multi-index
    in the shape of ["State","RegionName"].
    
    The resulting dataframe is 67x10730.
    '''
    
    return housing_data
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


In [40]:
def run_ttest():
    
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether we fail to reject the null hypothesis (false) or reject the null hypothesis (true)
    as well as the p-value of the confidence. 
    '''
    university_towns = get_list_of_university_towns()
    housing_data = convert_housing_data_to_quarters()

    unis = university_towns.to_records(index=False).tolist()

    uni_housing_data = university_towns.join(housing_data,on=['State','RegionName'],how='inner').set_index(['State','RegionName'])
    non_uni_housing_data = housing_data[~housing_data.index.isin(unis)]

    recession_start = get_recession_start()
    recession_bottom = get_recession_bottom()

    indx = uni_housing_data.columns.get_loc(recession_start)-1
    before_recession = uni_housing_data.columns[indx]

    uni_housing_data = uni_housing_data[[before_recession,recession_bottom]]
    non_uni_housing_data = non_uni_housing_data[[before_recession,recession_bottom]]

    uni_housing_data['Price Ratio'] = uni_housing_data[before_recession]/uni_housing_data[recession_bottom]
    non_uni_housing_data['Price Ratio'] = non_uni_housing_data[before_recession]/non_uni_housing_data[recession_bottom]


    ttest_statistic,p = ttest_ind(uni_housing_data['Price Ratio'],non_uni_housing_data['Price Ratio'],nan_policy='omit')
    different = str()


    if p<0.01:
        different = True
    else:
        different = False

    

    
    return (different,p)
run_ttest()

(True, 0.002724063704761164)